# Basic Impact-Z example


This is a basica example using the parameters described in:

C. Mayes, R. Ryne, D. Sagan
3D SPACE CHARGE IN BMAD
https://epaper.kek.jp/ipac2018/papers/thpak085.pdf

# Imports

In [ ]:
from impact import ImpactZ
import impact.z as IZ

from pmd_beamphysics.units import mec2, c_light
import numpy as np
import os

# Parameters

Set some parameters to be used in the input

In [ ]:
energy0 = 10e6
gamma0 = energy0 / mec2
betagamma0 = np.sqrt(gamma0**2 - 1)
sigma_x0 = 1e-3
sigma_y0 = 1e-3
sigma_z0 = 0.1e-3

beta_x0 = 1000
beta_y0 = 100
n_particle = 100_000

# Impact-Z input

The input object contains the header and lattice information.

In [ ]:
input = IZ.ImpactZInput(
    initial_particles=None,
    n_particle=n_particle,
    nx=32,
    ny=32,
    nz=32,
    distribution=IZ.DistributionType.gauss,
    twiss_beta_x=beta_x0,
    twiss_norm_emit_x=sigma_x0**2 * betagamma0 / beta_x0,
    twiss_beta_y=beta_y0,
    twiss_norm_emit_y=sigma_y0**2 * betagamma0 / beta_y0,
    average_current=0.13,
    reference_kinetic_energy=energy0 - mec2,
    reference_particle_mass=mec2,
    reference_particle_charge=-1.0,
    reference_frequency=1300000000.0,
    lattice=[
        IZ.WriteFull(name="initial_particles", file_id=100),
        IZ.Drift(
            name="MyDrift",
            length=1.0,
            steps=100,
            map_steps=100,
        ),
        IZ.WriteFull(name="final_particles", file_id=101),
    ],
)

A convenience function will set the longitudinal distribution from rms duration and energy spread:

In [ ]:
input.set_twiss_z(sigma_t=0.1e-3 / c_light, sigma_energy=1)

Here is a compact readback. Note that values that are default are not displayed

In [ ]:
input

# Create ImpactZ object from input

In [ ]:
I = ImpactZ(input)

# Run

In [ ]:
I.run()

# Plot

Plotting accepts any key that from the output statistics.

In [ ]:
I.plot(["sigma_x", "sigma_y"], y2="norm_emit_x")

# Extract particles

In [ ]:
P0 = I.output.particles["initial_particles"]
P1 = I.output.particles["final_particles"]

Check consistency with the initial particle sizes

In [ ]:
P0 = I.output.particles["initial_particles"]
(
    P0["sigma_x"] / sigma_x0,
    P0["sigma_y"] / sigma_y0,
    P0["sigma_t"] * c_light / sigma_z0,
    P0["mean_energy"] / energy0,
)

In [ ]:
P1.plot("delta_t", "energy")

# Try again with particles

In [ ]:
I2 = ImpactZ(input, initial_particles=P0)
I2.run()
P3 = I2.output.particles["initial_particles"]
P4 = I2.output.particles["final_particles"]

In [ ]:
P4.plot("delta_t", "energy")

# Archiving and readback

In [ ]:
I.archive("impactz.h5")

In [ ]:
I2 = ImpactZ.from_archive("impactz.h5")

I2.plot()

# Cleanup

In [ ]:
os.remove("impactz.h5")